In [25]:
import numpy as np

import wgpu
from wgpu.utils.compute import compute_with_buffers
%load_ext line_profiler

In [2]:
data_file = np.load("dct_data_jul11.npz")
dct = data_file['dct']
mask = dct != 0
dct -= np.amin(dct)
dct *= mask

In [3]:
dct[:8, :8]

array([[451, 223, 216, 214, 206,   0,   0,   0],
       [143, 191, 227, 209, 209,   0,   0,   0],
       [168, 208, 212, 209,   0,   0,   0,   0],
       [200,   0, 209,   0,   0,   0,   0,   0],
       [204,   0, 208,   0,   0,   0,   0,   0],
       [206,   0,   0,   0,   0,   0,   0,   0],
       [206,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0]], dtype=int16)

In [27]:
def read_shader_preview():
    return open("./rle_preview.wgsl").read()

def read_shader():
    return open("./rle.wgsl").read()

a = read_shader_preview()
b = read_shader()

In [5]:

# def generate_basis():
zigzag = np.array(
    [
        [0, 0],
        [0, 1],
        [1, 0],
        [2, 0],
        [1, 1],
        [0, 2],
        [0, 3],
        [1, 2],
        [2, 1],
        [3, 0],
        [4, 0],
        [3, 1],
        [2, 2],
        [1, 3],
        [0, 4],
        [0, 5],
        [1, 4],
        [2, 3],
        [3, 2],
        [4, 1],
        [5, 0],
        [6, 0],
        [5, 1],
        [4, 2],
        [3, 3],
        [2, 4],
        [1, 5],
        [0, 6],
        [0, 7],
        [1, 6],
        [2, 5],
        [3, 4],
        [4, 3],
        [5, 2],
        [6, 1],
        [7, 0],
        [7, 1],
        [6, 2],
        [5, 3],
        [4, 4],
        [3, 5],
        [2, 6],
        [1, 7],
        [2, 7],
        [3, 6],
        [4, 5],
        [5, 4],
        [6, 3],
        [7, 2],
        [7, 3],
        [6, 4],
        [5, 5],
        [4, 6],
        [3, 7],
        [4, 7],
        [5, 6],
        [6, 5],
        [7, 4],
        [7, 5],
        [6, 6],
        [5, 7],
        [6, 7],
        [7, 6],
        [7, 7],
    ]
)
zig_dim1 = zigzag[:, 0].astype(np.int32)
zig_dim2 = zigzag[:, 1].astype(np.int32)

In [6]:
zigzag.shape

(64, 2)

In [7]:
dtype_mapping = {
    np.int8: "b",
    np.int16: "h",
    np.int32: "i",
    np.float16: "e",
    np.float32: "f",
}

In [8]:
dispatch_counts = np.array([dct.shape[0] // 8, dct.shape[1] // 8]).astype(np.int16)

In [9]:
dct_flat = dct.flatten().astype(np.int32)

Logic: Run preview shader

In [29]:
import time
start_time = time.time()
def my_func():
    out_shape = ((512*512) / (64*64),)
    out = compute_with_buffers(
        input_arrays={0: dct_flat,
                     1: zig_dim1,
                     2: zig_dim2},
        output_arrays={3: (*out_shape, dtype_mapping[np.int32])},
        shader=a,
        n=(64,64, 1)
    )
    
    # print("done!")
    rle_lengths = np.frombuffer(out[3], dtype=np.int32)
    # print(f"rle length is {rle_lengths[:10]}")
    total_length = np.sum(rle_lengths)
    # print(f"total length is {total_length}")
    start_pts = np.zeros_like(rle_lengths)
    start_pts[1:] = np.cumsum(rle_lengths[:-1])
    # print(f"start_pts is {start_pts[:10]}")
    
    
    out = compute_with_buffers(
        input_arrays={0: dct_flat,
                      1: zig_dim1,
                      2:zig_dim2,
                      3: start_pts,
                      4: rle_lengths},
        
        output_arrays={5: (total_length, dtype_mapping[np.int32])},
        shader=b,
        n=(64,64,1)
    )
%lprun -f my_func my_func()

Timer unit: 1e-09 s

Total time: 0.0194585 s
File: /tmp/ipykernel_3190820/1076818565.py
Function: my_func at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def my_func():
     4         1       1118.0   1118.0      0.0      out_shape = ((512*512) / (64*64),)
     5         2   10386448.0    5e+06     53.4      out = compute_with_buffers(
     6         2       1538.0    769.0      0.0          input_arrays={0: dct_flat,
     7         1        289.0    289.0      0.0                       1: zig_dim1,
     8         1        309.0    309.0      0.0                       2: zig_dim2},
     9         1       3424.0   3424.0      0.0          output_arrays={3: (*out_shape, dtype_mapping[np.int32])},
    10         1        405.0    405.0      0.0          shader=a,
    11         1        273.0    273.0      0.0          n=(64,64, 1)
    12                                               )
    13                       